In [284]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import numpy as np
import seaborn as sns
from pathlib import Path
sns.set_style('whitegrid')  # Set whitegrid style from Seaborn
#import mplfinance as mpf
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "notebook"
import plotly.express as px
import matplotlib.ticker as tkr
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None
from collections import defaultdict
from scipy import stats
from datetime import datetime
import pytz
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 500)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import neptune as neptune
from scipy.interpolate import CubicSpline
from scipy.interpolate import interp1d
from scipy.interpolate import PchipInterpolator


In [285]:
# Load the datasets
actuals_df  = pd.read_csv('GR price actuals.csv')
provider1basic_df  = pd.read_csv('provider 1 GR price forecasts.csv')
provider1extra_df  = pd.read_csv('provider 1 GR price forecasts extra.csv')
provider2_df  = pd.read_csv('provider 2 GR price forecasts.csv')
provider3_df  = pd.read_csv('provider 3 GR price forecasts.csv')

actuals_df.drop(columns=['rownum'], inplace=True)

In [286]:
provider1basic_df

,datetime_from,value
0,2024-07-12 23:00:00.000 +0200,305.86
1,2024-07-12 22:00:00.000 +0200,362.43
2,2024-07-12 21:00:00.000 +0200,463.18
3,2024-07-12 20:00:00.000 +0200,482.93
4,2024-07-12 19:00:00.000 +0200,471.13
...,...,...
7604,2023-07-26 04:00:00.000 +0200,101.43
7605,2023-07-26 03:00:00.000 +0200,100.20
7606,2023-07-26 02:00:00.000 +0200,105.33
7607,2023-07-26 01:00:00.000 +0200,114.11


In [287]:
provider1extra_df

,Unnamed: 0,datetime_from,value
0,0,2024-03-27 08:00:00+01:00,74.99
1,1,2024-03-27 09:00:00+01:00,62.19
2,2,2024-03-27 10:00:00+01:00,52.03
3,3,2024-03-27 11:00:00+01:00,45.00
4,4,2024-03-27 12:00:00+01:00,42.96
5,5,2024-03-27 13:00:00+01:00,43.56
6,6,2024-03-27 14:00:00+01:00,46.74
7,7,2024-03-27 15:00:00+01:00,61.11
8,8,2024-03-27 16:00:00+01:00,74.73
9,9,2024-03-27 17:00:00+01:00,82.50


In [288]:
# Load and inspect each dataset individually
print("Provider 1 Basic DataFrame Head:\n", provider1basic_df.head())
print("Provider 1 Basic DataFrame Info:\n", provider1basic_df.info())

print("Provider 1 Extra DataFrame Head:\n", provider1extra_df.head())
print("Provider 1 Extra DataFrame Info:\n", provider1extra_df.info())

print("Provider 2 DataFrame Head:\n", provider2_df.head())
print("Provider 2 DataFrame Info:\n", provider2_df.info())

print("Provider 3 DataFrame Head:\n", provider3_df.head())
print("Provider 3 DataFrame Info:\n", provider3_df.info())

print("Actuals DataFrame Head:\n", actuals_df.head())
print("Actuals DataFrame Info:\n", actuals_df.info())

Provider 1 Basic DataFrame Head:
                    datetime_from   value
0  2024-07-12 23:00:00.000 +0200  305.86
1  2024-07-12 22:00:00.000 +0200  362.43
2  2024-07-12 21:00:00.000 +0200  463.18
3  2024-07-12 20:00:00.000 +0200  482.93
4  2024-07-12 19:00:00.000 +0200  471.13
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7609 entries, 0 to 7608
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   datetime_from  7609 non-null   object 
 1   value          7609 non-null   float64
dtypes: float64(1), object(1)
memory usage: 119.0+ KB
Provider 1 Basic DataFrame Info:
 None
Provider 1 Extra DataFrame Head:
    Unnamed: 0              datetime_from  value
0           0  2024-03-27 08:00:00+01:00  74.99
1           1  2024-03-27 09:00:00+01:00  62.19
2           2  2024-03-27 10:00:00+01:00  52.03
3           3  2024-03-27 11:00:00+01:00  45.00
4           4  2024-03-27 12:00:00+01:00  42.96
<class 'pandas.core.fra

In [289]:
print(actuals_df.isna().sum())
print(provider1basic_df.isna().sum())
print(provider2_df.isna().sum())
print(provider3_df.isna().sum())

datetime_from    0
price            0
dtype: int64
datetime_from    0
value            0
dtype: int64
datetime_from    0
value            0
dtype: int64
datetime_from    0
value            0
dtype: int64


#### Provider1 Integration

In [290]:
# Clean up the extra file by removing the 'Unnamed: 0' column if it exists
if 'Unnamed: 0' in provider1extra_df.columns:
    provider1extra_df = provider1extra_df.drop(columns=['Unnamed: 0'])

# Convert datetime columns to datetime objects for merging
provider1basic_df['datetime_from'] = pd.to_datetime(provider1basic_df['datetime_from'], utc=True)
provider1extra_df['datetime_from'] = pd.to_datetime(provider1extra_df['datetime_from'], utc=True)


# Merge the dataframes, prioritizing values from provider1extra_df and including all values from provider1extra_df
merged_df = pd.merge(provider1basic_df, provider1extra_df, on='datetime_from', how='outer', suffixes=('_basic', '_extra'))

# Replace basic values with extra values where they exist
merged_df['value'] = merged_df['value_extra'].combine_first(merged_df['value_basic'])

# Drop the extra columns used for merging
merged_df = merged_df.drop(columns=['value_basic', 'value_extra'])

print(merged_df.isna().sum())


# Ensure there are no NaNs (there shouldn't be as per the logic)
merged_df = merged_df.dropna()

# Sort by datetime to ensure chronological order
merged_df = merged_df.sort_values(by='datetime_from').reset_index(drop=True)

# Output the merged dataframe to a CSV file
merged_df.to_csv('updated_provider1_forecasts.csv', index=False)

# Print the first few rows of the merged dataframe
print(merged_df.head())

datetime_from    0
value            0
dtype: int64
              datetime_from   value
0 2023-07-25 22:00:00+00:00  116.47
1 2023-07-25 23:00:00+00:00  114.11
2 2023-07-26 00:00:00+00:00  105.33
3 2023-07-26 01:00:00+00:00  100.20
4 2023-07-26 02:00:00+00:00  101.43


In [291]:
merged_df.shape

(8080, 2)

In [292]:
merged_df.duplicated().sum()

0

In [293]:
provider1_df = merged_df

In [294]:
provider1_df

,datetime_from,value
0,2023-07-25 22:00:00+00:00,116.47
1,2023-07-25 23:00:00+00:00,114.11
2,2023-07-26 00:00:00+00:00,105.33
3,2023-07-26 01:00:00+00:00,100.20
4,2023-07-26 02:00:00+00:00,101.43
...,...,...
8075,2024-07-12 17:00:00+00:00,471.13
8076,2024-07-12 18:00:00+00:00,482.93
8077,2024-07-12 19:00:00+00:00,463.18
8078,2024-07-12 20:00:00+00:00,362.43


In [295]:
april_values = provider1_df[provider1_df['datetime_from'].dt.month == 4]
april_values.head(150)

,datetime_from,value
5658,2024-04-01 00:00:00+00:00,50.89
5659,2024-04-01 01:00:00+00:00,50.55
5660,2024-04-01 02:00:00+00:00,57.02
5661,2024-04-01 03:00:00+00:00,66.15
5662,2024-04-01 04:00:00+00:00,79.37
5663,2024-04-01 05:00:00+00:00,76.86
5664,2024-04-01 06:00:00+00:00,60.48
5665,2024-04-01 07:00:00+00:00,21.30
5666,2024-04-01 08:00:00+00:00,2.63
5667,2024-04-01 09:00:00+00:00,0.03


#### UTC Conversion , Timezone strip & Date Filtering 

In [296]:
# Convert datetime columns to UTC
actuals_df['datetime_from'] = pd.to_datetime(actuals_df['datetime_from'], utc=True)
provider1_df['datetime_from'] = pd.to_datetime(provider1_df['datetime_from'], utc=True)
provider2_df['datetime_from'] = pd.to_datetime(provider2_df['datetime_from'], utc=True)
provider3_df['datetime_from'] = pd.to_datetime(provider3_df['datetime_from'], utc=True)

# Strip away the timezone information
actuals_df['datetime_from'] = actuals_df['datetime_from'].dt.tz_localize(None)
provider1_df['datetime_from'] = provider1_df['datetime_from'].dt.tz_localize(None)
provider2_df['datetime_from'] = provider2_df['datetime_from'].dt.tz_localize(None)
provider3_df['datetime_from'] = provider3_df['datetime_from'].dt.tz_localize(None)

In [297]:
provider1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8080 entries, 0 to 8079
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   datetime_from  8080 non-null   datetime64[ns]
 1   value          8080 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 126.4 KB


In [298]:
# Filter the dataframes to keep only the dates between 28/07/2023 and 05/07/2024
start_date = pd.Timestamp('2023-07-28 00:00')
end_date = pd.Timestamp('2024-07-05 23:00')


filtered_actuals_df = actuals_df[(actuals_df['datetime_from'] >= start_date) & (actuals_df['datetime_from'] <= end_date)]
filtered_provider1_df = provider1_df[(provider1_df['datetime_from'] >= start_date) & (provider1_df['datetime_from'] <= end_date)]
filtered_provider2_df = provider2_df[(provider2_df['datetime_from'] >= start_date) & (provider2_df['datetime_from'] <= end_date)]
filtered_provider3_df = provider3_df[(provider3_df['datetime_from'] >= start_date) & (provider3_df['datetime_from'] <= end_date)]

In [299]:
# Renaming the value columns for clarity
filtered_provider1_df.rename(columns={'value': 'price1'}, inplace=True)
filtered_provider2_df.rename(columns={'value': 'price2'}, inplace=True)
filtered_provider3_df.rename(columns={'value': 'price3'}, inplace=True)

In [300]:
# Display the filtered dataframes before interpolation
print("Filtered Actuals DataFrame Head:\n", filtered_actuals_df.head())
print("Filtered Provider 1 DataFrame Head:\n", filtered_provider1_df.head())
print("Filtered Provider 2 DataFrame Head:\n", filtered_provider2_df.head())
print("Filtered Provider 3 DataFrame Head:\n", filtered_provider3_df.head())

Filtered Actuals DataFrame Head:
           datetime_from   price
650 2023-07-28 00:00:00  107.99
651 2023-07-28 01:00:00  106.04
652 2023-07-28 02:00:00  106.10
653 2023-07-28 03:00:00  105.00
654 2023-07-28 04:00:00  102.22
Filtered Provider 1 DataFrame Head:
          datetime_from  price1
50 2023-07-28 00:00:00   98.83
51 2023-07-28 01:00:00   95.19
52 2023-07-28 02:00:00   97.00
53 2023-07-28 03:00:00  100.36
54 2023-07-28 04:00:00  109.23
Filtered Provider 2 DataFrame Head:
           datetime_from  price2
153 2024-07-05 23:00:00  111.39
154 2024-07-05 22:00:00  118.82
155 2024-07-05 21:00:00  132.61
156 2024-07-05 20:00:00  148.59
157 2024-07-05 19:00:00  195.33
Filtered Provider 3 DataFrame Head:
           datetime_from   price3
153 2024-07-05 23:00:00   90.826
154 2024-07-05 22:00:00  100.837
155 2024-07-05 21:00:00   97.156
156 2024-07-05 20:00:00  112.625
157 2024-07-05 19:00:00  138.556


In [301]:
print("Filtered Actuals DataFrame NaN:\n", filtered_actuals_df.isna().sum())
print("Filtered Provider 1 DataFrame NaN:\n", filtered_provider1_df.isna().sum())
print("Filtered Provider 2 DataFrame NaN:\n", filtered_provider2_df.isna().sum())
print("Filtered Provider 3 DataFrame NaN:\n", filtered_provider3_df.isna().sum())

Filtered Actuals DataFrame NaN:
 datetime_from    0
price            0
dtype: int64
Filtered Provider 1 DataFrame NaN:
 datetime_from    0
price1           0
dtype: int64
Filtered Provider 2 DataFrame NaN:
 datetime_from    0
price2           0
dtype: int64
Filtered Provider 3 DataFrame NaN:
 datetime_from    0
price3           0
dtype: int64


In [302]:
# Merging the dataframes
merged_df = filtered_actuals_df.merge(filtered_provider1_df, on='datetime_from', how='outer')
merged_df = merged_df.merge(filtered_provider2_df, on='datetime_from', how='outer')
merged_df = merged_df.merge(filtered_provider3_df, on='datetime_from', how='outer')

In [303]:
merged_df.head()

,datetime_from,price,price1,price2,price3
0,2023-07-28 00:00:00,107.99,98.83,95.05,107.123
1,2023-07-28 01:00:00,106.04,95.19,93.70,104.801
2,2023-07-28 02:00:00,106.10,97.00,87.98,102.977
3,2023-07-28 03:00:00,105.00,100.36,89.45,109.143
4,2023-07-28 04:00:00,102.22,109.23,101.46,116.560


In [304]:
merged_df.isna().sum()

datetime_from      0
price             24
price1           392
price2             0
price3             0
dtype: int64

In [305]:
merged_df['datetime_from'] = pd.to_datetime(merged_df['datetime_from'])


### Spline Interpolation

In [306]:
# Generate 1-hour intervals from 28/07/2023 to 05/07/2024
start_date = pd.to_datetime('2023-07-28 00:00')
end_date = pd.to_datetime('2024-07-05 23:00')
date_range = pd.date_range(start=start_date, end=end_date, freq='H')

In [307]:
intervals_df = pd.DataFrame(date_range, columns=['datetime_from'])


In [308]:
merged = intervals_df.merge(merged_df, on='datetime_from', how='left')


In [309]:
interpolated_df = merged.copy()
interpolated_df[['price', 'price1', 'price2', 'price3']] = interpolated_df[['price', 'price1', 'price2', 'price3']].interpolate(method='pchip')

In [310]:
# Check if all hours are present
expected_hours = pd.date_range(start=start_date, end=end_date, freq='H')
actual_hours = interpolated_df['datetime_from']
missing_hours_final_check = expected_hours.difference(actual_hours)
missing_hours_final_check_count = len(missing_hours_final_check)

In [311]:
missing_hours_final_check_count

0

In [312]:
# interpolated_df.to_csv('final.csv')

In [313]:
interpolated_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8256 entries, 0 to 8255
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   datetime_from  8256 non-null   datetime64[ns]
 1   price          8256 non-null   float64       
 2   price1         8256 non-null   float64       
 3   price2         8256 non-null   float64       
 4   price3         8256 non-null   float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 387.0 KB


In [314]:
# Count the number of entries for each hour in the merged dataframe
interpolated_df['hour'] = interpolated_df['datetime_from'].dt.hour

hourly_counts = interpolated_df.groupby('hour').count()

# Select relevant columns for the counts
hourly_counts = hourly_counts[['datetime_from', 'price', 'price1', 'price2', 'price3']]
hourly_counts.columns = ['total_count', 'actuals_count', 'provider1_count', 'provider2_count', 'provider3_count']

# Display the hourly counts summary
print(hourly_counts)

      total_count  actuals_count  provider1_count  provider2_count  \
hour                                                                 
0             344            344              344              344   
1             344            344              344              344   
2             344            344              344              344   
3             344            344              344              344   
4             344            344              344              344   
5             344            344              344              344   
6             344            344              344              344   
7             344            344              344              344   
8             344            344              344              344   
9             344            344              344              344   
10            344            344              344              344   
11            344            344              344              344   
12            344   

In [315]:
interpolated_df.tail()

,datetime_from,price,price1,price2,price3,hour
8251,2024-07-05 19:00:00,162.83,143.01,195.33,138.556,19
8252,2024-07-05 20:00:00,130.50,115.74,148.59,112.625,20
8253,2024-07-05 21:00:00,109.16,113.48,132.61,97.156,21
8254,2024-07-05 22:00:00,126.43,106.10,118.82,100.837,22
8255,2024-07-05 23:00:00,118.13,103.51,111.39,90.826,23
